cube cipher
###########

## Challenge
The Rubik's Cube is well known as a challenging puzzle with quintillions of possible configurations.

Why not make a cipher out of it?

```ncat --ssl cube-cipher.challs.pwnoh.io 1337```

## Included files
* cube_cipher.c
* cube_cipher.h

## Analysis
Well it sure has been a long time since I've seen the word "nibble", being half a byte.

When the program starts, the flag is loaded to memory, and the algorithm is loaded from the disk. A cube is built from the flag string, and the algorithm is executed on it.

The server presents a menu with several options:
1. Execute an algorithm
2. Display cube
3. Display cube as bytes
4. Re-apply cube cipher
5. Exit

### Algorithm
An algorithm is a string comprising of \[FRLUDBMESxyz]*

In [ ]:
from pwn import *
conn = remote("cube-cipher.challs.pwnoh.io", 1337, ssl=True)
print(conn.recvuntilS("Option: "))

conn.sendline(b"2")
print(conn.recvuntilS("Option: "))

conn.sendline(b"3")
print(conn.recvuntilS("Option: "))

conn.sendline(b"5")
print(conn.recvallS())

[x] Opening connection to cube-cipher.challs.pwnoh.io on port 1337
[x] Opening connection to cube-cipher.challs.pwnoh.io on port 1337: Trying 2600:1f16:75:1c01::4
[+] Opening connection to cube-cipher.challs.pwnoh.io on port 1337: Done
Welcome to the Interactive Cube Cipher App!
Try and break my cipher! (you can't)
Options:
	1: Execute an algorithm
	2: Display cube
	3: Display cube as bytes
	4: Re-apply cube cipher
	5: Exit
Option: 
            5  0  6
            6  8  2
            7  6 15

15  0  5    5  4  6    0  3 11
 6  5  7    2 10  3    3  6  7
 3  5  0    4  7  3    6  5  5

            5 15  0
            7 12  9
            5  6  6

            4  3  9
            0  7  0
           13  3  5
Option: 
5462a347303b36765550668276ff056573505f07c9566439070d35
Option: 


I'd hazard a guess that the goal would be to identify what the algorithm is. By repeated application of the algorithm, one ought to be able to determine the mappings.

In [ ]:
conn = remote("cube-cipher.challs.pwnoh.io", 1337, ssl=True)
conn.recvuntilS("Option: ")

conn.sendline(b"3")
original_line = previous_line = conn.recvlineS()

candidate_positions = {}
for i in range(48):
    candidate_positions[i] = list(range(48))

conn.recvuntilS("Option: ")

for i in range(10):
    conn.sendline(b"4")
    conn.recvlineS()
    conn.recvuntilS("Option: ")

    conn.sendline(b"3")
    new_line = conn.recvlineS()
    conn.recvuntilS("Option: ")

    for fromPos in range(48):
        for candidateTo in candidate_positions[fromPos]:
            if previous_line[fromPos] != new_line[candidateTo]:
                candidate_positions[fromPos].remove(candidateTo)
    previous_line = new_line
    print(new_line)

    print(candidate_positions)

[x] Opening connection to cube-cipher.challs.pwnoh.io on port 1337
[x] Opening connection to cube-cipher.challs.pwnoh.io on port 1337: Trying 2600:1f16:75:1c01::4
[+] Opening connection to cube-cipher.challs.pwnoh.io on port 1337: Done


/usr/lib/python3/dist-packages/pwnlib/tubes/tube.py:1679: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  return packing._decode(func(self, *a, **kw))


635272b57d6078355563666433f5007c35656750594743f00a690f

{0: [1, 2, 4, 6, 7, 9, 11, 13, 15, 16, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 38, 40, 42, 44, 46], 1: [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 42, 44, 46], 2: [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 21, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46], 3: [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47], 4: [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47], 5: [1, 3, 5, 7, 9, 11, 13, 14, 16, 18, 19, 21, 23, 24, 25, 27, 29, 31, 32, 34, 36, 38, 40, 42, 44, 45, 47], 6: [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 42, 44, 46], 7: [1, 3, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 37, 39, 41, 43, 45, 47], 8: [1, 3, 5, 7, 9, 11, 13, 14, 16, 18, 19, 21, 23, 24, 25, 27, 29, 31, 32, 34, 36, 38, 40, 42, 44, 45, 47], 9: [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 2

In [35]:
originalString = ["X"] * 48

for fromPos in range(48):
    toPos = candidate_positions[fromPos]
    if len(toPos) == 1:
        originalString[fromPos] = original_line[toPos[0]]

print(''.join(originalString))

6263X4667b7468655f6X556233Xf706c3479535f593X557X


Although there are some nibbles missing from this string, there is enough to reassemble it by inspection

```
62 63 X4 66 7b 74 68 65 5f 6X 55 62 33 Xf 70 6c 34 79 53 5f 59 3X 55 7X
b  c  ?  f  {  t  h  e  _  ?  U  b  3  ?  p  l  4  y  S  _  Y  ?  U  ?

62 63 74 66 7b 74 68 65 5f 63 55 62 33 5f 70 6c 34 79 53 5f 59 30 55 7D
b  c  t  f  {  t  h  e  _  C  U  b  3  _  p  l  4  y  S  _  Y  0  U  }
```

The flag is
bctf{the_cUb3_pl4yS_Y0U}